## Daily Monte Carlo Block Bootstrap

In [1]:

"""
Macro Monte Carlo for XRP/USDT (1D x 1000 bars) — no argparse version
- ดึง OHLCV 1D ย้อนหลัง 1000 แท่งจาก Binance ด้วย ccxt
- คำนวณ log-returns จากราคาปิด (ไม่ใช้ TA)
- Block-bootstrap Monte Carlo (คงโครงสร้างในบล็อก)
- สรุปความน่าจะเป็นแตะกรอบ $2 / $4, percentile ของ min/max ในอนาคต
- แนะนำกริดจากแถบเปอร์เซ็นไทล์ (บังคับ step $0.10 และจำกัดจำนวนชั้น)

ติดตั้ง: pip install ccxt pandas numpy
"""

import ccxt
import numpy as np
import pandas as pd
from datetime import datetime, timezone

# ===================== CONFIG =====================
CONFIG = {
    "symbol": "JTO/USDT",
    "timeframe": "1d",
    "lookback_bars": 1000,     # ใช้แท่งล่าสุด 1000 แท่ง
    "paths": 10000,            # จำนวนเส้นทาง MC
    "horizon_bars": 60,        # ระยะมองไปข้างหน้า (วัน)
    "block_len": 24,           # ความยาวบล็อกสำหรับ bootstrap
    "band_percentiles": (10, 90),  # ใช้ p10 ของ minimum และ p90 ของ maximum
    "lower_level": 1.54,       # กรอบ Macro ด้านล่างที่อยากประเมิน P(touch)
    "upper_level": 4.10,       # กรอบ Macro ด้านบนที่อยากประเมิน P(touch)
    "grid_min_step": 0.10,     # บังคับระยะห่างไมโคร $0.10
    "max_layers": 12,          # จำกัดจำนวนชั้นกริดรวม (ขึ้น+ลง)
    "seed": 42                 # เพื่อความ reproducible; ตั้งเป็น None ถ้าไม่ต้องการ
}
# ==================================================


def fetch_ohlcv_1d_binance(symbol: str, lookback_bars: int) -> pd.DataFrame:
    ex = ccxt.binance({"enableRateLimit": True})
    # ดึงล่าสุด 1000 แท่งตรง ๆ
    raw = ex.fetch_ohlcv(symbol, timeframe="1d", limit=lookback_bars)
    if not raw or len(raw) < 50:
        raise RuntimeError("Not enough OHLCV returned.")
    df = pd.DataFrame(raw, columns=["ts", "open", "high", "low", "close", "volume"])
    df["dt"] = pd.to_datetime(df["ts"], unit="ms", utc=True).dt.tz_convert("Asia/Bangkok")
    return df[["dt", "open", "high", "low", "close", "volume"]]


def block_bootstrap_returns(ret: np.ndarray, horizon: int, block_len: int, paths: int, rng: np.random.Generator):
    """
    สุ่มบล็อกของ log-returns จากอดีตมาต่อ ๆ กันจนได้ความยาวตาม horizon
    ret: 1D numpy array ของ log-returns ประวัติ
    return: ndarray shape = (paths, horizon) ของ simulated log-returns
    """
    n = len(ret)
    if n < block_len + 5:
        raise ValueError("Return history too short for chosen block_len.")
    out = np.empty((paths, horizon), dtype=float)

    # indices ที่เริ่มบล็อกได้ (ให้มีที่ว่าง block_len)
    valid_starts = np.arange(0, n - block_len + 1)

    for p in range(paths):
        path = []
        while len(path) < horizon:
            start = rng.choice(valid_starts)
            blk = ret[start:start + block_len]
            path.extend(blk.tolist())
        out[p, :] = np.array(path[:horizon])
    return out


def simulate_price_paths(spot: float, logret_paths: np.ndarray) -> np.ndarray:
    """
    แปลง log-returns → เส้นทางราคา (geometric compounding จาก spot)
    logret_paths: (paths, horizon)
    return prices: (paths, horizon)
    """
    cum = np.cumsum(logret_paths, axis=1)
    prices = spot * np.exp(cum)
    return prices


def prob_touch_level(paths: np.ndarray, level: float, side: str) -> float:
    """
    side='down' → เคยลงต่ำกว่า level ไหม
    side='up'   → เคยขึ้นสูงกว่า level ไหม
    """
    if side == "down":
        touched = (paths.min(axis=1) <= level)
    else:
        touched = (paths.max(axis=1) >= level)
    return float(touched.mean()) * 100.0


def summarize_macro(prices: np.ndarray, spot: float, lower: float, upper: float, p_lo=10, p_hi=90):
    """
    สรุป percentiles ของ min/max ในช่วงอนาคต และ P(touch) ระดับสำคัญ
    """
    path_min = prices.min(axis=1)
    path_max = prices.max(axis=1)

    p_touch_low = prob_touch_level(prices, lower, "down")
    p_touch_high = prob_touch_level(prices, upper, "up")

    # percentiles
    pct = [10, 50, 90]
    min_pct = np.percentile(path_min, pct).tolist()
    max_pct = np.percentile(path_max, pct).tolist()

    band_low = np.percentile(path_min, p_lo)
    band_high = np.percentile(path_max, p_hi)

    return {
        "spot": spot,
        "p_touch_low": p_touch_low,
        "p_touch_high": p_touch_high,
        "min_pct": min_pct,
        "max_pct": max_pct,
        "band_low": float(band_low),
        "band_high": float(band_high)
    }


def suggest_grid(spot: float, band_low: float, band_high: float, step: float, max_layers: int):
    """
    สร้างแผนกริดคร่าว ๆ จาก band (p10_min → p90_max) ด้วย step บังคับ
    - จำกัดจำนวนชั้นรวม (ขึ้น+ลง) ตาม max_layers
    """
    # เลเยอร์ฝั่งลง
    dn_levels = []
    price = spot - step
    while price >= band_low + 1e-9:
        dn_levels.append(round(price, 4))
        price -= step

    # เลเยอร์ฝั่งขึ้น
    up_levels = []
    price = spot + step
    while price <= band_high - 1e-9:
        up_levels.append(round(price, 4))
        price += step

    # จำกัดจำนวนชั้นรวม
    total = len(dn_levels) + len(up_levels)
    if total > max_layers:
        # ตัดแบบสัดส่วน (ให้ล่างมากกว่านิด ๆ ก็ได้)
        # ที่ง่ายสุด: ตัดปลายทั้งสองจนเหลือรวม = max_layers โดยสลับตัด
        keep_dn = len(dn_levels)
        keep_up = len(up_levels)
        while (keep_dn + keep_up) > max_layers:
            if keep_dn >= keep_up and keep_dn > 0:
                keep_dn -= 1
            elif keep_up > 0:
                keep_up -= 1
            else:
                break
        dn_levels = dn_levels[:keep_dn]
        up_levels = up_levels[:keep_up]

    return dn_levels[::-1], up_levels  # เรียงล่าง→ใกล้ spot ก่อนสำหรับฝั่งลง


def export_macro_csv(csv_filename: str, cfg: dict, summ: dict, path_min: np.ndarray, path_max: np.ndarray):
    """
    เขียน CSV แบบรวม:
      1) ส่วน path-level: path_id, min_price, max_price
         (คอลัมน์สรุปทั้งหมดจะเป็น NaN)
      2) ต่อด้วยส่วนสรุป 1 แถว: spot, p_touch_low, p_touch_high, band_low, band_high,
         min_p10/min_p50/min_p90, max_p10/max_p50/max_p90 และ metadata พื้นฐาน

    ไฟล์เดียว อ่านง่ายต่อ pipeline ที่คาด schema เดิม
    """
    # --- 1) Path-level ---
    paths_df = pd.DataFrame({
        "path_id": np.arange(len(path_min), dtype=int),
        "min_price": path_min.astype(float),
        "max_price": path_max.astype(float),
    })

    # เติมคอลัมน์สรุป (เป็น NaN สำหรับส่วน path)
    summary_cols = {
        "spot": np.nan,
        "p_touch_low": np.nan,
        "p_touch_high": np.nan,
        "band_low": np.nan,
        "band_high": np.nan,
        "min_p10": np.nan, "min_p50": np.nan, "min_p90": np.nan,
        "max_p10": np.nan, "max_p50": np.nan, "max_p90": np.nan,
        # optional metadata
        "symbol": None,
        "timeframe": None,
        "lookback_bars": np.nan,
        "paths": np.nan,
        "horizon_bars": np.nan,
        "block_len": np.nan,
        "band_lo_pct": np.nan,
        "band_hi_pct": np.nan,
        "lower_level": np.nan,
        "upper_level": np.nan,
        "generated_at_gmt7": None,
    }
    for k in summary_cols:
        paths_df[k] = summary_cols[k]

    # --- 2) Summary 1 row ---
    summ_df = pd.DataFrame([{
        "path_id": np.nan,
        "min_price": np.nan,
        "max_price": np.nan,
        "spot": float(summ["spot"]),
        "p_touch_low": float(summ["p_touch_low"]),
        "p_touch_high": float(summ["p_touch_high"]),
        "band_low": float(summ["band_low"]),
        "band_high": float(summ["band_high"]),
        "min_p10": float(summ["min_pct"][0]),
        "min_p50": float(summ["min_pct"][1]),
        "min_p90": float(summ["min_pct"][2]),
        "max_p10": float(summ["max_pct"][0]),
        "max_p50": float(summ["max_pct"][1]),
        "max_p90": float(summ["max_pct"][2]),
        # metadata (ช่วยให้ reproducible/debug ได้)
        "symbol": cfg["symbol"],
        "timeframe": cfg["timeframe"],
        "lookback_bars": int(cfg["lookback_bars"]),
        "paths": int(cfg["paths"]),
        "horizon_bars": int(cfg["horizon_bars"]),
        "block_len": int(cfg["block_len"]),
        "band_lo_pct": int(cfg["band_percentiles"][0]),
        "band_hi_pct": int(cfg["band_percentiles"][1]),
        "lower_level": float(cfg["lower_level"]),
        "upper_level": float(cfg["upper_level"]),
        "generated_at_gmt7": datetime.now(timezone.utc).astimezone().strftime('%Y-%m-%d %H:%M:%S %Z'),
    }])

    out_all = pd.concat([paths_df, summ_df], ignore_index=True)
    out_all.to_csv(csv_filename, index=False)
def export_macro_csv(csv_filename: str, cfg: dict, summ: dict, path_min: np.ndarray, path_max: np.ndarray):
    """
    เขียน CSV แบบรวม:
      1) ส่วน path-level: path_id, min_price, max_price
         (คอลัมน์สรุปทั้งหมดจะเป็น NaN)
      2) ต่อด้วยส่วนสรุป 1 แถว: spot, p_touch_low, p_touch_high, band_low, band_high,
         min_p10/min_p50/min_p90, max_p10/max_p50/max_p90 และ metadata พื้นฐาน

    ไฟล์เดียว อ่านง่ายต่อ pipeline ที่คาด schema เดิม
    """
    # --- 1) Path-level ---
    paths_df = pd.DataFrame({
        "path_id": np.arange(len(path_min), dtype=int),
        "min_price": path_min.astype(float),
        "max_price": path_max.astype(float),
    })

    # เติมคอลัมน์สรุป (เป็น NaN สำหรับส่วน path)
    summary_cols = {
        "spot": np.nan,
        "p_touch_low": np.nan,
        "p_touch_high": np.nan,
        "band_low": np.nan,
        "band_high": np.nan,
        "min_p10": np.nan, "min_p50": np.nan, "min_p90": np.nan,
        "max_p10": np.nan, "max_p50": np.nan, "max_p90": np.nan,
        # optional metadata
        "symbol": None,
        "timeframe": None,
        "lookback_bars": np.nan,
        "paths": np.nan,
        "horizon_bars": np.nan,
        "block_len": np.nan,
        "band_lo_pct": np.nan,
        "band_hi_pct": np.nan,
        "lower_level": np.nan,
        "upper_level": np.nan,
        "generated_at_gmt7": None,
    }
    for k in summary_cols:
        paths_df[k] = summary_cols[k]

    # --- 2) Summary 1 row ---
    summ_df = pd.DataFrame([{
        "path_id": np.nan,
        "min_price": np.nan,
        "max_price": np.nan,
        "spot": float(summ["spot"]),
        "p_touch_low": float(summ["p_touch_low"]),
        "p_touch_high": float(summ["p_touch_high"]),
        "band_low": float(summ["band_low"]),
        "band_high": float(summ["band_high"]),
        "min_p10": float(summ["min_pct"][0]),
        "min_p50": float(summ["min_pct"][1]),
        "min_p90": float(summ["min_pct"][2]),
        "max_p10": float(summ["max_pct"][0]),
        "max_p50": float(summ["max_pct"][1]),
        "max_p90": float(summ["max_pct"][2]),
        # metadata (ช่วยให้ reproducible/debug ได้)
        "symbol": cfg["symbol"],
        "timeframe": cfg["timeframe"],
        "lookback_bars": int(cfg["lookback_bars"]),
        "paths": int(cfg["paths"]),
        "horizon_bars": int(cfg["horizon_bars"]),
        "block_len": int(cfg["block_len"]),
        "band_lo_pct": int(cfg["band_percentiles"][0]),
        "band_hi_pct": int(cfg["band_percentiles"][1]),
        "lower_level": float(cfg["lower_level"]),
        "upper_level": float(cfg["upper_level"]),
        "generated_at_gmt7": datetime.now(timezone.utc).astimezone().strftime('%Y-%m-%d %H:%M:%S %Z'),
    }])

    out_all = pd.concat([paths_df, summ_df], ignore_index=True)
    out_all.to_csv(csv_filename, index=False)




def main():
    cfg = CONFIG.copy()
    rng = np.random.default_rng(cfg["seed"]) if cfg["seed"] is not None else np.random.default_rng()

    print(f"[i] Fetching {cfg['symbol']} {cfg['timeframe']} OHLCV from Binance (last {cfg['lookback_bars']} bars)...")
    df = fetch_ohlcv_1d_binance(cfg["symbol"], cfg["lookback_bars"])
    close = df["close"].to_numpy(dtype=float)

    spot = float(close[-1])
    logret = np.diff(np.log(close))

    print(f"[i] Spot: {spot:.6f}")
    print(f"[i] Running block-bootstrap Monte Carlo: paths={cfg['paths']:,}, horizon_bars={cfg['horizon_bars']}, block_len={cfg['block_len']}")
    
    # สุ่มเส้นทาง log-returns แล้วแปลงเป็นราคา
    log_paths = block_bootstrap_returns(
        ret=logret,
        horizon=cfg["horizon_bars"],
        block_len=cfg["block_len"],
        paths=cfg["paths"],
        rng=rng
    )
    price_paths = simulate_price_paths(spot, log_paths)

    # สรุปผล Macro
    summ = summarize_macro(
        price_paths, spot,
        lower=cfg["lower_level"],
        upper=cfg["upper_level"],
        p_lo=CONFIG["band_percentiles"][0],
        p_hi=CONFIG["band_percentiles"][1]
    )

    band_low = summ["band_low"]
    band_high = summ["band_high"]
    width = band_high - band_low

    # สร้างกริดจาก band ด้วย step บังคับ
    dn_levels, up_levels = suggest_grid(
        spot=spot,
        band_low=band_low,
        band_high=band_high,
        step=cfg["grid_min_step"],
        max_layers=cfg["max_layers"]
    )

    # === บันทึก CSV รวม path-level + summary ===
    export_macro_csv(
        csv_filename="macro_montecarlo.csv",
        cfg=cfg,
        summ=summ,
        path_min=price_paths.min(axis=1),
        path_max=price_paths.max(axis=1),
    )
    print(f"\n[i] บันทึกผลลัพธ์ลงไฟล์ CSV แล้ว: macro_montecarlo.csv")

    # ========= แสดงผล =========
    print("\n=== ผลลัพธ์ Macro Monte Carlo (1D, ใช้ประวัติ 1000 แท่ง) ===")
    print(f" ราคาปัจจุบัน (Spot): {summ['spot']:.6f}")
    print(f" ความน่าจะเป็น 'ลงไปแตะ' ต่ำกว่า {cfg['lower_level']:.2f}: {summ['p_touch_low']:.2f}%")
    print(f" ความน่าจะเป็น 'ขึ้นไปแตะ' สูงกว่า {cfg['upper_level']:.2f}: {summ['p_touch_high']:.2f}%")
    print(f" ค่าต่ำสุดในช่วงจำลอง (เปอร์เซ็นไทล์ p10/p50/p90): {np.round(summ['min_pct'], 6).tolist()}")
    print(f" ค่าสูงสุดในช่วงจำลอง (เปอร์เซ็นไทล์ p10/p50/p90): {np.round(summ['max_pct'], 6).tolist()}")
    
    print(f"\n[แถบคาดการณ์จากเปอร์เซ็นไทล์]  p{CONFIG['band_percentiles'][0]}_min ไป p{CONFIG['band_percentiles'][1]}_max")
    print(f" ช่วงราคา: {band_low:.6f} ไป {band_high:.6f}  (ความกว้าง {width:.6f})")
    
    print("\n[ข้อเสนอการวางกริด]")
    print(f" ระยะห่างขั้น (step) = ${cfg['grid_min_step']:.2f} | จำนวนชั้นสูงสุด = {cfg['max_layers']}")
    print(f" ชั้นฝั่งลง ({len(dn_levels)}): {dn_levels}")
    print(f" ชั้นฝั่งขึ้น ({len(up_levels)}): {up_levels}")
    print(f"\nเสร็จสิ้นเมื่อ (GMT+7): {datetime.now(timezone.utc).astimezone().strftime('%Y-%m-%d %H:%M:%S %Z')}")



if __name__ == "__main__":
    main()




[i] Fetching JTO/USDT 1d OHLCV from Binance (last 1000 bars)...
[i] Spot: 1.661000
[i] Running block-bootstrap Monte Carlo: paths=10,000, horizon_bars=60, block_len=24

[i] บันทึกผลลัพธ์ลงไฟล์ CSV แล้ว: macro_montecarlo.csv

=== ผลลัพธ์ Macro Monte Carlo (1D, ใช้ประวัติ 1000 แท่ง) ===
 ราคาปัจจุบัน (Spot): 1.661000
 ความน่าจะเป็น 'ลงไปแตะ' ต่ำกว่า 1.54: 85.43%
 ความน่าจะเป็น 'ขึ้นไปแตะ' สูงกว่า 4.10: 1.09%
 ค่าต่ำสุดในช่วงจำลอง (เปอร์เซ็นไทล์ p10/p50/p90): [0.896699, 1.270727, 1.593093]
 ค่าสูงสุดในช่วงจำลอง (เปอร์เซ็นไทล์ p10/p50/p90): [1.699793, 2.091319, 2.927268]

[แถบคาดการณ์จากเปอร์เซ็นไทล์]  p10_min ไป p90_max
 ช่วงราคา: 0.896699 ไป 2.927268  (ความกว้าง 2.030569)

[ข้อเสนอการวางกริด]
 ระยะห่างขั้น (step) = $0.10 | จำนวนชั้นสูงสุด = 12
 ชั้นฝั่งลง (6): [1.061, 1.161, 1.261, 1.361, 1.461, 1.561]
 ชั้นฝั่งขึ้น (6): [1.761, 1.861, 1.961, 2.061, 2.161, 2.261]

เสร็จสิ้นเมื่อ (GMT+7): 2025-09-22 08:45:31 UTC
